In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# CHEKPOINT - Regresión lineal

Vamos a trabajar con un conjunto de datos sobre alquileres de bicicletas que fue utilizado en un concurso de Kaggle. Se proporcionan datos de alquiler por hora que abarcan dos años. 

**_Queremos predecir el número total de bicicletas alquiladas utilizando sólo la información disponible en el set de entrenamiento._**


**CAMPOS DEL DATASET**

- **datetime** - dia y hora - timestamp
- **season** -  1 = primavera, 2 = verano, 3 = otoño, 4 = invierno
- **holiday** - feriado
- **workingday** - si es dia laborable (1) o fin de semana (0)
- **weather** - 
      1: Clear, Few clouds
      2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
      3: Light Snow, Light Rain + Thunderstorm + Scattered clouds,Light Rain + Scattered clouds
      4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

- **temp** - temperatura en Celsius
- **atemp** - sensación térmica
- **humidity** - humedad
- **windspeed** - velocidad del viento
- **casual** - ususarios no-registrados (number of non-registered user rentals initiated)
- **registered** - ususarios registrados (number of registered user rentals initiated)
- **count** - total de alquileres


Notemos que este dataset contiene variables contínuas y categóricas, veremos más adelante como trabajar con ambos tipos. Así mismo, observamos que existen variables que por su composición no pueden utilizarse para predecir el resultado. Este es el caso de los datos bajo las columnas 'casual' y 'registered' que entre ambas suman el resultado a predecir. 

In [ ]:
# Importar libreria de pandas 
# Leer los datos de bikeshare.csv y setear datetime como índice. 
import pandas as pd
bikes = pd.read_csv('../Data/bikes.csv', index_col='datetime', parse_dates=True)

In [ ]:
# Observar las dimensiones del dataframe
bikes.shape

In [ ]:
# Tomar una muestra de 10 observaciones (usar el método sample)
bikes.sample(10)

In [ ]:
# Dado que "count" es un método de pandas, cambiar el nombre de la columna "count" y renombrarla por "total":
bikes.rename(columns={'count':'total'}, inplace=True)

In [ ]:
bikes.sample(2)

# Regresión Lineal

Construir un modelo en **[Scikit-Learn](https://scikit-learn.org/stable/)** lleva 5 pasos.

1.   Preparar los datos en una **matriz de features y un array target**.

2.   Elegir una clase de modelo importando la **clase de estimador** apropiado de Scikit-Learn.

3.   Seleccionar los **hiperparámetros** del modelo **instanciando** la clase con los valores deseados

4.   Ajustar el modelo a los datos invocando el método **fit()** de la instancia del modelo.

5.   Aplicar el modelo a **nuevos datos**:


Empezamos por una regresión lineal simple!

## Regresión Lineal Simple

La regresión Lineal Simple es cuando utilizamos una unica variable para predecir la variable target. En nuestro ejemplo vamos a elegir trabajar con la Temperatura como variable predictora.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Definimos parámetros globales para matplotlib.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 16

In [ ]:
# Realizar un scatterplot con 'temp' sobre el 'eje x' y 'total' sobre el 'eje y'
bikes.plot(kind='scatter', x='temp', y='total', alpha=0.2);

In [ ]:
# En adelante usaremos un Array 'features_cols' para almacenar los features que usaremos para entrenar el modelo.
feature_cols = []

In [ ]:
# instanciar a feature_cols con 'temp' como único argumento
# A continuación crear el arrays de variables independientes X utilizando feature_cols como mascara de nuestro dataframe
# y finalmente generar el array de la variable dependiente y utilizando la columna 'total'
feature_cols = ['temp']
X = bikes[feature_cols]
y = bikes['total']

In [ ]:
# Importamos paquete, instanciamos el estimador y fiteamos el modelo ("classic" sklearn!)

from sklearn.linear_model import LinearRegression

In [ ]:
# Instanciamos el modelo

linreg = LinearRegression()

In [ ]:
# Entrenar el modelo con todo el dataset (no usaremos train-test-split en esta oportunidad)

linreg.fit(X, y)

In [ ]:
# Imprimir los coeficientes y el intercept del modelo entrenado

print (linreg.intercept_)
print (linreg.coef_)

Interpretación del  **intercepto ** ($\beta_0$):

- Es el valor esperado de $ y $ cuando $ x $ = 0.

Interpretación del coeficiente de ** "temp" ** ($\beta_1$):

- Es el cambio en $ y $ dividido por cambio en $ x $, o la "pendiente".
- Así, un aumento de la temperatura de 1 grado Celsius está ** asociado con ** un aumento de alquiler de 9.17 bicicletas.
 $ \beta_1 $ sería **negativo** si un aumento en la temperatura se asociara con una **disminución** en los alquileres.

### Usando el modelo para predecir

¿Cuántos alquileres de bicicletas podríamos predecir si la temperatura era de 25 grados Celsius?

In [ ]:
# calcular total de bicis alquiladas utilizando la formula de la recta con los paramétros calculados anteriormente
# y = b0 + b1*x 

temp= 25

linreg.intercept_ + linreg.coef_*temp

In [ ]:
# Realizar el mismo calculo pero realizando la prediccion con el modelo que entrenamos.
# nota: deberán convertir la temperatura de entero a un np.array ya que el modelo trabaja con np.arrays 
#(importar numpy)

import numpy as np

test_sklearn = np.array(temp).reshape(-1,1)
linreg.predict(test_sklearn)

## Regresión Lineal Multiple

In [ ]:
# Exploramos más features
feature_cols = ['temp', 'weather', 'humidity']

In [ ]:
# matriz de correlación (rangos de 1 a -1)
bikes[feature_cols].corr()

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando a heatmap
sns.heatmap(bikes[feature_cols].corr(), vmin=-1, vmax=1, center=0, cmap="YlGnBu");

In [ ]:
# Al igual que lo hicimos para regresion lineal simple, crear nuevamente X (utilizando el nuevo feature_cols) e y 
# Luego instanciar el modelo y entrenarlo (nuevamente sin utlizar train-test-split).
# nota: instanciar el modelo bajo el nombre: 'linreg'
# Finalmente imprimir los coeficientes del nuevo modelo.
X = bikes[feature_cols]
y = bikes.total

#instanciamos y entrenamos el modelo
linreg = LinearRegression()
linreg.fit(X, y)

# Imprimimos coeficientes
print (linreg.intercept_)
print (linreg.coef_)

In [ ]:
## para observarlo mejor miramos el nombre con el coeficiente
list(zip(feature_cols, linreg.coef_))

## Eligiendo entre modelos

### Métricas de evaluación para problemas de regresión

 Hay tres métricas de evaluación comunes para problemas de regresión:

**El error absoluto medio** (MAE) es la media del valor absoluto de los errores:

$$ \frac 1n\sum_ {i = 1}^n |y_i-\hat{y}_i| $$

**Error cuadratico medio** (MSE) es la media de los errores al cuadrado:

$$ \frac 1n\sum_ {i = 1}^n(y_i- \hat{y}_i)^2 $$

**Raiz del Error cuadrático medio** (RMSE) es la raíz cuadrada de la media de los errores al cuadrado:

$$ \sqrt{\frac 1n\sum_{i = 1}^n(y_i- \hat{y}_i)^2} $$

todas estas son **funciones de pérdida**, porque queremos minimizarlas.

In [ ]:
# Ejemplo de valores de respuesta verdaderos y predichos
true = [10, 7, 5, 5, 10, 8, 8, 15, 12]
pred = [12, 8, 3, 4, 12, 9, 8, 12, 13]

In [ ]:
# ¿Calculamos métricas a mano? ¡Por supuesto que no!

from sklearn import metrics
import numpy as np
print ('MAE:', metrics.mean_absolute_error(true, pred))
print ('MSE:', metrics.mean_squared_error(true, pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(true, pred)))
print ('R2:', metrics.r2_score(true, pred))

## Comparando modelos usando sets de entrenamiento/test y RMSE

In [ ]:
# Definir una función y llamarla 'train_test_error' que acepte como parámetro una lista de features (feature_cols), 
# que genere la matriz de variables independentes 'X' y el array de la variable target 'y' 
# que luego haga el split entre train y test reservando un 25% de las observaciones para testeo, 
# y finalmente imprima los errores MAE, MSE, RMSE y R2

from sklearn.model_selection import train_test_split

def train_test_error(feature_cols):
    X = bikes[feature_cols]
    y = bikes.total
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    
    
    print (feature_cols)
    print ("y_test sample: ",y_test.values[0:10])
    print ("y_pred sample: ",y_pred[0:10].astype(int))
    print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('R2:', metrics.r2_score(y_test, y_pred))
    print ()
    return 

In [ ]:
# utilizando la función 'train_test_error' recien creada, comparamos diferentes ensambles de features.
# probar primero con feature_cols = ['temp', 'humidity']
# y luego con feature_cols = ['temp', 'humidity','windspeed']
# Qué observaciones pueden hacer sobre los resultados?

train_test_error(['temp', 'humidity'])
train_test_error(['temp', 'humidity','windspeed'])

# Creando Features

## Manejo de variables categóricas

Scikit-learn espera que todos los features sean numéricos. Entonces, ¿cómo incluimos una característica categórica en nuestro modelo?

- ** Categorías ordenadas: ** transformarlas en valores numéricos que representan ese orden 
- ** Categorías no ordenadas: ** utilizar codificación ficticia (0/1) (Variables-dummy)

¿Cuáles son las características categóricas de nuestro conjunto de datos?

- ** Categorías ordenadas: ** weather (ya codificado con valores numéricos que representan el orden)
- ** Categorías no ordenadas: ** season (necesita variables dummy), holiday (Ya está codificada como dummy), workingday (ya está codificada como dummy)

Para la estación, no podemos simplemente dejar la codificación como 1 = primavera, 2 = verano, 3 = otoño y 4 = invierno, porque eso implicaría una ** relación ordenada **. En cambio, creamos ** variables dummies: **

In [ ]:
# crear variables dummies
season_dummies = pd.get_dummies(bikes.season, prefix='season')

# imprimimos para ver 5 filas cualquieras
season_dummies.sample(n=5, random_state=1)

Nota: El método `get_dummies` puede recibir un dataframe entero, en ese caso no modifica las variables numéricas y genera dummies para todas las categóricas que encuentre. 

Sin embargo, en realidad sólo necesitamos ** tres variables dummy (no cuatro) **, y por lo tanto vamos a dropear la primera variable dummy. 
¿Por qué?

In [ ]:
# Eliminamos la primer columna ya que es redundante
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)

# imprimimos 5 filas cualquieras
season_dummies.sample(n=5, random_state=1)

En general, si se tiene una feature categórica con ** k valores posibles **, se tienen que crear ** k-1 variables dummies **.

In [ ]:
# Concatenar el DataFrame original y el dummy DataFrame (axis = 0 significa filas, axis = 1 significa columnas)
bikes = pd.concat([bikes, season_dummies], axis=1)

# imprimimos 5 filas cualquieras
bikes.sample(n=5)

In [ ]:
# utilizamos ahora la funcion train_test_error que definimos anteriormente con 
# los ultimos features seleccionados (feature_cols = ['temp','humidity','windspeed', 'season_2', 'season_3', 'season_4'])

train_test_error(['temp','humidity','windspeed', 'season_2', 'season_3', 'season_4'])

Se observa una leve mejora en el R2, pero no muy significativa.

### Continuamos con feature engineering
Crear las siguientes features:

- **hora:** como una variable dummy (de 0 a 23), 
- **día:** como una feature categórica (día = 1 de 7am a 8pm y día = 0 de lo contrario)

In [ ]:
# Generar variables dummy a partir de hora
bikes['hora'] = bikes.index.hour
bikes_dummies = pd.get_dummies(bikes.index.hour, drop_first=True)
bikes_dummies.index = bikes.index

In [ ]:
bikes_dummies.head()


In [ ]:
# Concatenar las variables dummy al dataframe original
bikes  = pd.concat([bikes,bikes_dummies], axis = 1)

In [ ]:
# Crear días como categórica
dia = (bikes.index.hour >= 7) & (bikes.index.hour <= 20)
dia

In [ ]:
# Agregar la columna de dia
bikes['dia']= dia.astype(int)
bikes['dia'].sample(5)

In [ ]:
# Por ultimo corremos nuestra función train_test_error sobre el nuevo set de features:
# ['temp','humidity','windspeed', 'season_2', 'season_3', 'season_4',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

train_test_error(['temp','humidity','windspeed', 'season_2', 'season_3', 'season_4','dia',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])

## Conclusiones: 

Ventajas de la regresión lineal:

- Simple de explicar
- Muy interpretable
- El entrenamiento y predicción de modelos son rápidos
- Es invariante a cambios en la escala de los features. 


Desventajas de la regresión lineal:

- El rendimiento es (generalmente) no competitivo con los mejores métodos de aprendizaje supervisado debido a un alto sesgo